In [20]:
from neo4j import GraphDatabase

uri = "neo4j://10.195.2.227:7687"
username = "neo4j"
password = "neo4j-connect"

driver = GraphDatabase.driver(uri, auth=(username, password))

def add_node(session, node):
    result = session.run("CREATE (a:Wiki $node)", node=node)
        
def add_edge(session, docid1, docid2, edge):
    result = session.run("MATCH (a:Wiki {docid: $docid1}) MATCH (b:Wiki {docid: $docid2}) MERGE (a)-[:$edge]->(b)", docid1=docid1, docid2=docid2, edge=edge)

In [21]:
import json
from tqdm import tqdm
import time
import gc

s_time = time.time()
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))
with driver.session() as session:
    for d in tqdm(data):
        entry = {
            'docid': d['docid'],
            'title': d['title'],
            'text': d['text'],
            'uri': d['uri']
        }
        add_node(session, entry)
        
print(f"Time for file {file} is {int(e_time - time.time())}sec")
del data
gc.collect()

  6%|████▍                                                                         | 567/10000 [00:43<12:00, 13.09it/s]


BufferError: Existing exports of data: object cannot be re-sized

In [50]:
data = json.load(open("C:\\Users\\96181\\Desktop\\Master's Thesis\\T-Rex-Sample-Data.json", "r+"))

In [56]:
def add_edge(session, docid1, docid2, edge):
    edge_name = edge['predicate_name'].replace(' ', '_')
    result = session.run(f"MATCH (a:Wiki {{docid: $docid1}}) MATCH (b:Wiki {{docid: $docid2}}) MERGE (a)-[r:{edge_name}]->(b) SET r += $edge", docid1=docid1, docid2=docid2, edge=edge)

predicate_names = {}
def get_predicate_name(uri):
    prop_id = uri.split('/')[-1]
    if prop_id in predicate_names:
        return predicate_names[prop_id]
    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
    response = requests.get(url)
    label = response.json()['entities'][prop_id]['labels']['en']['value']
    predicate_names[prop_id] = label
    return label

with driver.session() as session:
    t = data[0]['triples'][0]
    predicate_name = get_predicate_name(t['predicate']['uri'])
    entry = {
        'predicate_surfaceform': t['predicate']['surfaceform'],
        'predicate_uri': t['predicate']['uri'],
        'predicate_name': predicate_name,
        'object_surfaceform': t['object']['surfaceform'],
        'subject_surfaceform': t['subject']['surfaceform'],  
    }
    add_edge(session, t['subject']['uri'], t['object']['uri'], entry)

In [61]:
def get_predicate_name(uri):
    prop_id = uri.split('/')[-1]
    if prop_id in predicate_names:
        return predicate_names[prop_id]
    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
    response = requests.get(url)
    label = response.json()['entities'][prop_id]['labels']['en']['value']
    predicate_names[prop_id] = label
    return label

In [48]:
import requests

prop_id = 'P31'
url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={prop_id}&format=json"
response = requests.get(url)
label = response.json()['entities'][prop_id]['labels']['en']['value']
label

'instance of'

In [19]:
text = """text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard. Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore "k'onalah", or the code of ethics in Chechen culture."""

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import sent_tokenize
from sklearn.cluster import AgglomerativeClustering

# Split the text into sentences
sentences = sent_tokenize(text)

# Vectorize the sentences using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

# Use hierarchical clustering to chunk sentences
clustering = AgglomerativeClustering(n_clusters=2).fit(X.toarray())
labels = clustering.labels_

# Group sentences by cluster for final chunks
chunks = {}
for sentence, label in zip(sentences, labels):
    if label in chunks:
        chunks[label].append(sentence)
    else:
        chunks[label] = [sentence]

for label, chunk_sentences in chunks.items():
    print(' '.join(chunk_sentences))
    print()

text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard.

Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore "k'onalah", or the code of ethics in Chechen culture.



In [49]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re

def merged_sentence(words):
    if type(words[0]) == str:
        sentence = ' '.join(words)
        sentence = re.sub(r' ([''`"'']),', r'\1,', sentence)
        sentence = re.sub(r'\s([.,;:!?])', r'\1', sentence)
        sentence = re.sub(r'\s([\'])\s', r'\1', sentence)
        sentence = re.sub(r'\s([(])\s', r' \1', sentence)
        sentence = re.sub(r'\s([)])\s', r'\1 ', sentence)
        return sentence
    else:
        return ' '.join([merged_sentence(w) for w in words])

SENTENCE_SIZE = 124
OVERLAP_SIZE = 32
def chunk_text(text):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_chunk_size = 0
    for sentence in sentences:
        words = word_tokenize(sentence)
        # Estimate if adding the next sentence will exceed the token limit
        if current_chunk_size + len(words) > SENTENCE_SIZE:
            # Add current_chunk to chunks and start a new chunk
            chunks.append(merged_sentence(current_chunk))
            overlap = current_chunk[-1][-OVERLAP_SIZE:] if len(current_chunk[-1]) > OVERLAP_SIZE else current_chunk[-1]  # Get last 64 characters or full chunk
            current_chunk = [overlap, words] # Overlap
            current_chunk_size = len(overlap) + len(words)
        else:
            current_chunk.append(words)  # Add the next sentence to the current chunk
            current_chunk_size += len(words)
    chunks.append(merged_sentence(current_chunk))  # Add the last remaining chunk
    return chunks

chunk_text(text)

['text: Khas-Magomed Hadjimuradov (Chechen: Хас-Магомед Хаджимурадов) (born December 18, 1953 in Kazakhstan) is a Chechen nationalist bard. Khas-Magomed was born in the Zhambyl Province of Kazakhstan to Chechen parents, who had been forcibly removed from their homes in Chechnya as the result of the forced deportations of the Chechen and Ingush people to Central Asia on February 23, 1944. He spent much of his early youth in the village of Panfilovka in the Chuy Province of neighboring Kyrgyzstan, before his family returned their native village of Novye Atagi. After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975.',
 "After graduating from secondary school in 1971, he served in the Red Army from 1973 to 1975. In order to bolster the national identity of the Chechen people, Khas-Magomed performed nationalistic songs of Chechen heroes on Soviet television, hoping to revive the national consciousness of the Chechens and to restore `` k'onalah '', or the